In [1]:
from utils.data_loader import DataLoader

In [3]:
loader = DataLoader()

X_train = loader.subset_labels(num_labels=10, set_type='train')
X_val = loader.subset_labels(num_labels=10, set_type='val')

NameError: name 'Path' is not defined

In [ ]:
def multi_modal_cnn():
    with mirrored_strategy.scope():

        # Inputs
        patch_input = layers.Input(shape=(256, 256, 6), dtype='float32')
        tabular_input = layers.Input(shape=(29), dtype='float32')  

        # Augment data
    #     augmented = data_augmentation_for_visualization(patch_input)

        # From Scratch model
        x = layers.Conv2D(64, (3, 3), activation='relu')(patch_input)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Conv2D(128, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Conv2D(256, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((1, 1), padding='same')(x)
        x = layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)      

        # Add Dense layers for images
        x = layers.Flatten()(x)
        x = layers.Dense(256, activation='relu')(x)

        # Add Dense layers for Tabular data
        y = layers.Dense(512, activation='relu')(tabular_input)
        y = layers.Dense(256, activation='relu')(y)

        # Concatenate Image and tabular weights
        z = layers.Concatenate(axis=1)([x, y])

        # Add Classification Head
        z = layers.Dense(128, activation='relu')(z)
        classifier = layers.Dense(NUM_CLASSES, name='outputs', activation='softmax')(z)

        # Define inputs and outputs
        model = tf.keras.Model(inputs=[patch_input, tabular_input], outputs=classifier)

        # Optimizer
        optimizer = tfa.optimizers.AdamW(learning_rate=LEARNING_RATE, 
                                         weight_decay=weight_decay_rate)
        # Compile model
        model.compile(optimizer=optimizer,
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=[
                          tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                          tf.keras.metrics.SparseTopKCategoricalAccuracy(10, name="top-10-accuracy")
                      ]
                      )

In [ ]:
model.summary()

In [ ]:
class Patches_Generator_CNN(tf.keras.utils.Sequence) :
  
    def __init__(self, obs_ids, labels, batch_size) :
        self.obs_ids = obs_ids
        self.labels = labels
        self.batch_size = batch_size
        
        # to make the generator thread safe 
        self.lock = threading.Lock()

    def __len__(self) :
        return (np.floor(len(self.obs_ids) / float(self.batch_size))).astype(int)
  
    # returns one batch
    def __getitem__(self, idx) :
        X_batch = list()
        X_env_batch = list()
        y_batch = list()

        for i in range(idx * self.batch_size, (idx+1) * self.batch_size):
            if i >= len(self.obs_ids): break
            
            rgb, near_ir, landcover, altitude = load_patch(self.obs_ids[i], DATA_PATH)

            ni = near_ir.reshape(256, 256, 1)
            lc = landcover.reshape(256, 256, 1)
            alt = altitude.reshape(256, 256, 1)

            patch = np.concatenate((rgb, ni, lc, alt), axis=2)
            
            X_batch.append(patch)
            y_batch.append(self.labels[i])
            
            X_env_batch.append(tabular_train.loc[self.obs_ids[i]].values)
            
        with self.lock:
            return {'input_1': np.asarray(X_batch), 'input_2': np.asarray(X_env_batch)}, np.asarray(np.array(y_batch))

In [ ]:
train_data = Patches_Generator_CNN(X_train_idx, y_train, BATCH_SIZE)
val_data = Patches_Generator_CNN(X_val_idx, y_val, BATCH_SIZE)

# converting our train dataset to tf.data.Dataset
tf_train_dataset = tf.data.Dataset.from_generator(
    lambda: train_data ,  # Our generator 
    output_types = ({'input_1': tf.float32 , 'input_2': tf.float32}, tf.float32) , # How we're expecting our output dtype
#     output_shapes = ({'input_1': [BATCH_SIZE, 256 , 256, 6], 'input_2': [BATCH_SIZE, 29]} , [BATCH_SIZE, ]) # How we're expecting our output shape
)

tf_val_dataset = tf.data.Dataset.from_generator(
    lambda: val_data , 
    output_types = ({'input_1': tf.float32 , 'input_2': tf.float32}, tf.float32),
#     output_shapes = ({'input_1': [BATCH_SIZE, 256 , 256, 6], 'input_2': [BATCH_SIZE, 29]} , [BATCH_SIZE, ]) 
)

In [ ]:
cnn_results = model.fit(
    tf_train_dataset.repeat(),
    steps_per_epoch=np.floor(len(y_train)/BATCH_SIZE)*4,
    validation_data=tf_val_dataset.repeat(),
    validation_steps=np.floor(len(y_val)/BATCH_SIZE)*4,
    epochs=NUM_EPOCHS,
    callbacks=callbacks,
    max_queue_size=PRE_FETCH_NUM_BATCHES, 
    workers=NUM_THREADS, 
    use_multiprocessing=True
)